# Solar Underperformance Mini-Case (7-day)

This notebook detects and quantifies a PV site underperformance event by comparing **Actual (5-min)** vs **Expected/Day-Ahead (hourly)** generation.

**Outputs:** daily performance dashboard, automatic event isolation, 5‑minute drilldown, and ops-ready impact summary (duration + MWh lost).


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 140)


In [ ]:
# === Repo-friendly paths (works in Colab + local) ===
# In Colab: upload the project zip to the session, then run:
#   !unzip -o solar-underperformance-mini-case_READY_SUBMIT.zip
RAW_DIR = "data/raw"
ACTUAL_FILE   = "Actual_41.35_-72.95_2006_UPV_74MW_5_Min.csv"     # 5-min actual
EXPECTED_FILE = "DA_41.35_-72.95_2006_UPV_74MW_60_Min.csv"        # hourly expected (day-ahead)

os.makedirs(RAW_DIR, exist_ok=True)
print("RAW_DIR:", RAW_DIR)
print("Files in data/raw:", os.listdir(RAW_DIR))


## 1) Load data

- **Actual**: 5‑minute measurements (MW)
- **Expected (DA)**: hourly forecast (MW)

If the CSVs aren’t present, the notebook generates a synthetic sample so it still runs end-to-end.

In [ ]:
def _try_read_csv(path: str):
    return pd.read_csv(path) if os.path.exists(path) else None

def generate_synthetic_sample():
    """Fallback synthetic data (only used if raw CSVs are missing)."""
    np.random.seed(7)
    start = pd.Timestamp("2026-01-01 00:00:00")
    idx = pd.date_range(start, periods=7*24*12, freq="5min")
    h = (idx.hour + idx.minute/60).to_numpy(dtype=float)

    expected = np.clip(np.sin((h - 6) / 12 * np.pi), 0, 1) * 35
    expected = expected * (1 + np.random.normal(0, 0.03, len(idx)))
    expected = np.clip(expected, 0, None)

    actual = expected * (1 + np.random.normal(0, 0.05, len(idx)))

    # outage-like drop (midday)
    outage_day = pd.Timestamp("2026-01-04").date()
    mask_out = (idx.date == outage_day) & (h >= 11) & (h <= 15)
    actual[mask_out] = actual[mask_out] * 0.2

    # missing data window
    miss_day = pd.Timestamp("2026-01-06").date()
    mask_miss = (idx.date == miss_day) & (h >= 12) & (h <= 13)
    actual[mask_miss] = np.nan

    act_df = pd.DataFrame({"LocalTime": idx.strftime("%m/%d/%y %H:%M"), "Power(MW)": actual})
    exp_df = pd.DataFrame({"LocalTime": idx[::12].strftime("%m/%d/%y %H:%M"), "Power(MW)": expected[::12]})
    return act_df, exp_df, "synthetic"

def load_data():
    act_path = os.path.join(RAW_DIR, ACTUAL_FILE)
    exp_path = os.path.join(RAW_DIR, EXPECTED_FILE)

    act = _try_read_csv(act_path)
    exp = _try_read_csv(exp_path)

    if act is None or exp is None:
        print("⚠️ Raw CSVs not found. Using synthetic data so the notebook runs end-to-end.")
        return generate_synthetic_sample()

    return act, exp, "real_csv"

act_raw, exp_raw, source = load_data()
print("Source:", source)
print("Actual rows:", len(act_raw), "Expected rows:", len(exp_raw))
act_raw.head()


### Why do you see 0 MW values?

Nighttime solar production is naturally ~0 MW, so **both Actual and Expected are 0** for many timestamps (e.g., 00:00–06:xx). We avoid nighttime bias by focusing most KPIs/anomaly logic on **daylight periods where Expected > 0**.

In [ ]:
# Sanity check: show first non-zero expected hour (daylight start)
tmp_exp = exp_raw.copy()
tmp_exp["LocalTime"] = pd.to_datetime(tmp_exp["LocalTime"], format="%m/%d/%y %H:%M", errors="coerce")
print(tmp_exp[tmp_exp["Power(MW)"] > 0].head(3))


## 2) Build a unified 5-min fact table

We resample hourly Expected → 5‑minute to align with Actual, then compute:
- `gap_mw = expected - actual`
- `gap_mw_pos = max(gap_mw, 0)` (only underproduction)
- `ratio = actual/expected` (only when expected>0)

We keep:
- `fact` = full timeline (keeps missing flags)
- `fact_valid` = rows where actual exists (safe for ratios/energy)

In [ ]:
def to_ts(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["ts"] = pd.to_datetime(df["LocalTime"], format="%m/%d/%y %H:%M", errors="coerce")
    df = df.dropna(subset=["ts"])
    return df.set_index("ts").sort_index()

act = to_ts(act_raw)
exp = to_ts(exp_raw)

expected_5 = exp["Power(MW)"].resample("5min").interpolate("time")

fact = pd.DataFrame({
    "site_id": "SITE_001",
    "actual_mw": act["Power(MW)"],
    "expected_mw": expected_5,
})

# Keep fact as the full table (DO NOT drop rows here)
fact["flag_missing"] = fact["actual_mw"].isna()

# Treat missing expected as 0 (common at edges/night)
fact["expected_mw"] = fact["expected_mw"].fillna(0)

fact["gap_mw"] = fact["expected_mw"] - fact["actual_mw"]
fact["gap_mw_pos"] = fact["gap_mw"].clip(lower=0)

# Ratio (safe: NaN where expected=0)
fact["ratio"] = np.where(
    fact["expected_mw"] > 0,
    fact["actual_mw"] / fact["expected_mw"],
    np.nan
)

# Separate table for calculations that require actual values
fact_valid = fact.dropna(subset=["actual_mw"]).copy()

dim_site = pd.DataFrame([{
    "site_id": "SITE_001",
    "capacity_mw": 74,
    "data_source": source
}])

print("fact rows:", len(fact), "| fact_valid rows:", len(fact_valid))
fact.head(), dim_site


## 3) Daylight-only performance (EDA)

We filter to **daylight** points (Expected > 0.5 MW) and compute daily MWh and ratios.

> Why 0.5 MW? It removes nights and very-low edge values that can make ratios noisy.

In [ ]:
# Daylight view for KPI calculations (requires actual values)
fact_daylight = fact_valid[fact_valid["expected_mw"] > 0.5].copy()
print("Daylight rows:", len(fact_daylight), "of", len(fact))

fact_daylight[["actual_mw", "expected_mw", "gap_mw", "gap_mw_pos", "ratio"]].describe().T


In [ ]:
def mwh_from_5min(power_mw: pd.Series) -> float:
    # MW at 5-min resolution → MWh: sum(MW) * (5/60)
    return float(np.nansum(power_mw) * 5 / 60)

daily = fact_daylight.resample("1D").apply(lambda x: pd.Series({
    "actual_mwh": mwh_from_5min(x["actual_mw"]),
    "expected_mwh": mwh_from_5min(x["expected_mw"]),
    "lost_mwh": mwh_from_5min(x["gap_mw_pos"]),
}))
daily["ratio"] = np.where(daily["expected_mwh"] > 0, daily["actual_mwh"] / daily["expected_mwh"], np.nan)

display(daily)

# ---- Plots (Daily Dashboard) ----
plt.figure(figsize=(12,4))
plt.plot(daily.index, daily["expected_mwh"], marker="o", label="Expected (MWh)")
plt.plot(daily.index, daily["actual_mwh"], marker="o", label="Actual (MWh)")
plt.title("Daily Expected vs Actual Energy")
plt.ylabel("MWh/day")
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(12,3))
plt.plot(daily.index, daily["ratio"], marker="o", label="Actual/Expected")
plt.axhline(0.85, linestyle="--", label="0.85 triage threshold")
plt.title("Daily Performance Ratio")
plt.ylabel("Ratio")
plt.legend()
plt.tight_layout()
plt.show()


### Daily finding (add your narrative)

At the daily level, actual energy tracks expected energy on 6/7 days, indicating normal operation and forecast alignment. On 2026-01-04, actual generation drops sharply while expected remains stable, driving the performance ratio to ~0.61 (below the 0.85 triage threshold). This isolates a single high-severity underperformance event suitable for root-cause drilldown (outage/derate/curtailment vs data gap), which I investigate next at 5-minute resolution.


## 4) 5‑minute drilldown + anomaly detection

**Goal:** identify the exact underperformance window and quantify impact.

Practical rule (robust to partial derates):
- Daylight window: 08:00–18:00
- High-expected condition: `expected_mw >= 30`
- Underperform condition: `actual_mw <= min(ABS_FLOOR, REL_DROP * expected_mw)`
- Confirmed if sustained for `MIN_RUN_PTS` consecutive 5‑min points (e.g., 6 → 30 minutes)


In [ ]:
# Pick the worst day by daily ratio (automatic)
EVENT_DAY = daily["ratio"].idxmin().date()
print("Selected EVENT_DAY:", EVENT_DAY)

hrs = fact_daylight.index.hour + fact_daylight.index.minute/60

# ---- Rule thresholds (tunable) ----
ABS_FLOOR = 8      # MW absolute floor for "too low"
REL_DROP  = 0.30   # or <= 30% of expected
EXP_THR   = 30     # expected must be high (MW) to consider it meaningful
MIN_RUN_PTS = 6    # 6 * 5min = 30 min sustained

midday = (hrs >= 10) & (hrs <= 15)

fact_daylight["flag_outage_like"] = (
    midday
    & (fact_daylight["expected_mw"] >= EXP_THR)
    & (
        (fact_daylight["actual_mw"] <= ABS_FLOOR)
        | (fact_daylight["actual_mw"] <= REL_DROP * fact_daylight["expected_mw"])
    )
)

# Consecutive-run length (streak counter)
run = 0
run_len = []
for v in fact_daylight["flag_outage_like"].to_numpy():
    run = run + 1 if v else 0
    run_len.append(run)

fact_daylight["outage_run_len"] = run_len
fact_daylight["flag_outage_confirmed"] = fact_daylight["outage_run_len"] >= MIN_RUN_PTS

# Event window snapshot (08:00–18:00 on EVENT_DAY)
mask_event_window = (fact_daylight.index.date == EVENT_DAY) & (hrs >= 8) & (hrs <= 18)
cols = ["actual_mw","expected_mw","gap_mw","gap_mw_pos","ratio","flag_outage_like","outage_run_len","flag_outage_confirmed"]
display(fact_daylight.loc[mask_event_window, cols].head(80))


In [ ]:
# Confirmed event window + impact summary (ops-ready)
confirmed = fact_daylight.loc[
    (fact_daylight.index.date == EVENT_DAY) & (fact_daylight["flag_outage_confirmed"]),
    :
].copy()

print("Confirmed points:", len(confirmed))
print("Approx duration (minutes):", len(confirmed) * 5)

if len(confirmed) > 0:
    start_ts = confirmed.index.min()
    end_ts   = confirmed.index.max()
    print("Confirmed window:", start_ts, "→", end_ts)

    # Energy impact during confirmed window (MW * 5min → MWh)
    lost_mwh = float((confirmed["gap_mw_pos"].sum()) * 5/60)
    print("Estimated lost energy (MWh):", round(lost_mwh, 2))

    avg_ratio = float(np.nanmean(confirmed["ratio"]))
    print("Avg Actual/Expected ratio (confirmed window):", round(avg_ratio, 3))
else:
    print("No confirmed underperformance window found (try loosening thresholds).")


In [ ]:
# Plot: expected vs actual (5-min) and highlight confirmed underperformance
hrs = fact_daylight.index.hour + fact_daylight.index.minute/60
mask_event = (fact_daylight.index.date == EVENT_DAY) & (hrs >= 8) & (hrs <= 18)
event = fact_daylight.loc[mask_event].copy()

plt.figure(figsize=(12,4))
plt.plot(event.index, event["expected_mw"], label="Expected (MW)")
plt.plot(event.index, event["actual_mw"], label="Actual (MW)")

event_conf = event[event["flag_outage_confirmed"]]
if len(event_conf) > 0:
    plt.scatter(event_conf.index, event_conf["actual_mw"], s=18, label="Confirmed underperformance")

plt.title(f"5-min Drilldown: Expected vs Actual — {EVENT_DAY}")
plt.ylabel("MW")
plt.xlabel("Time")
plt.legend()
plt.tight_layout()
plt.show()


## 5) Interpretation & next steps

**What the drilldown shows:** Expected stays high around midday while Actual collapses to a low plateau for multiple hours, then recovers.

**Most likely explanations (needs ops/SCADA confirmation):**
- Curtailment command (grid or site-level setpoint)
- Inverter/transformer derate or trip (partial plant availability)
- Soiling/snow unlikely for a single-day sharp step change; shading unlikely at midday
- Data issue less likely because values are plausible (not NaN/flatline at 0), but confirm telemetry health

**What I would request next:**
- SCADA alarms / inverter availability and derate flags for the confirmed window
- Curtailment signals / dispatch instructions
- Irradiance / weather proxy to separate resource vs equipment
